In [2]:
import csv
import numpy as np
import pandas as pd

df = pd.read_csv('transactions.csv')
df.head()

,transaction_date,transaction_amount,contribution_or_distribution,commitment_id
0,01/01/2019,"$ (10,000.00)",contribution,1
1,01/01/2019,"$ (10,000.00)",contribution,2
2,01/01/2019,"$ (20,000.00)",contribution,3
3,01/01/2019,"$ (40,000.00)",contribution,4
4,01/01/2019,"$ (50,000.00)",contribution,5


In [4]:
def clean_amount(amount_str):
    # Remove dollar signs and spaces
    clean_str = amount_str.strip().replace('$', '').replace(' ', '')
    
    # Check if the value is in parentheses (negative)
    is_negative = '(' in clean_str and ')' in clean_str
    
    # Remove parentheses and commas, then convert to float
    clean_str = clean_str.replace('(', '').replace(')', '').replace(',', '')
    amount = float(clean_str)
    
    # Make negative if in parentheses
    return -amount if is_negative else amount

df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%m/%d/%Y')
df['transaction_amount'] = df['transaction_amount'].apply(clean_amount)
df.head()

,transaction_date,transaction_amount,contribution_or_distribution,commitment_id
0,2019-01-01,-10000.0,contribution,1
1,2019-01-01,-10000.0,contribution,2
2,2019-01-01,-20000.0,contribution,3
3,2019-01-01,-40000.0,contribution,4
4,2019-01-01,-50000.0,contribution,5


In [6]:
irr = 0.08

def get_npv(value, rate, days):
    return value * (1+rate)**(days/365)

get_npv(100, 0.08, 365)

108.0

In [15]:
def return_of_capital(commitment_id, date):
    """
    returns true if return of capital is satisfied
    TRUE - capital is paid back
    FALSE - capital is not yet paid back

    (capital returned?, additional capital for investor, capital for next tier )
    
    """

    total_commitment = df[
        (df['commitment_id'] == commitment_id) &
        (df['transaction_date'] < date) &
        (df['contribution_or_distribution'] == 'contribution')
    ]['transaction_amount'].sum()

    total_distribution = df[
        (df['commitment_id'] == commitment_id) &
        (df['transaction_date'] < date) &
        (df['contribution_or_distribution'] == 'distribution')
    ]['transaction_amount'].sum()

    max_distribution = np.min([np.abs(total_commitment), np.abs(total_distribution)])

    return (total_distribution + total_commitment >= 0), max_distribution, 0, total_distribution - max_distribution

roc_completed, roc_lp_allocation, roc_gp_allocation, next_tier_capital = return_of_capital(1, '2024-01-01')
print('Return of Capital Completed:', roc_completed)
print('LP Allocation:', np.abs(roc_lp_allocation))
print('GP Allocation:', roc_gp_allocation)
print('Next Tier Capital:', next_tier_capital)

Return of Capital Completed: True
LP Allocation: 50000.0
GP Allocation: 0
Next Tier Capital: 150000.0


In [ ]:
def preferred_return(commitment_id, date):

    roc_completed, roc_lp_allocation, roc_gp_allocation, next_tier_capital = return_of_capital(commitment_id, date)

    if not roc_completed:
        print('Return of Capital not completed')
        return False, 0, 0, 0

    date = np.datetime64(date)

    subset_df =  df[
        (df['commitment_id'] == commitment_id) &
        (df['transaction_date'] < date)
        # (df['contribution_or_distribution'] == 'contribution')
    ]

    final_distribution_date = subset_df['transaction_date'].values[-1]

    contribution_npv = 0
    distribution_npv = 0

    for (transaction_date, transaction_amount, contribution_or_distribution, commitment_id) in subset_df.values:
        if contribution_or_distribution == 'contribution':
            contribution_npv += get_npv(transaction_amount, irr, (final_distribution_date - transaction_date).days)
        else:
            distribution_npv += get_npv(transaction_amount, irr, (final_distribution_date - transaction_date).days)
    
    max_lp_allocation = np.min([np.abs(contribution_npv) - np.abs(roc_lp_allocation)])
    effective_lp_allocation = np.min([max_lp_allocation, next_tier_capital])

    return (next_tier_capital - max_lp_allocation >= 0), effective_lp_allocation, 0, next_tier_capital - effective_lp_allocation

pref_return_completed, pref_lp_allocation, pref_gp_allocation, next_tier_capital = preferred_return(1, '2024-01-02')
print('Preferred Return Completed:', pref_return_completed)
print('LP Allocation:', np.abs(pref_lp_allocation))
print('GP Allocation:', pref_gp_allocation)
print('Next Tier Capital:', next_tier_capital)

Preferred Return Completed: True
LP Allocation: 9606.613387258287
GP Allocation: 0
Next Tier Capital: 140393.3866127417


In [ ]:
carried_interest_rate = 0.2
catch_up_rate = 1

def catch_up(commitment_id, date):

    roc_completed, roc_lp_allocation, roc_gp_allocation, roc_capital_remaining = return_of_capital(commitment_id, date)
    pref_return_completed, pref_lp_allocation, pref_gp_allocation, pref_return_capital_remaining = preferred_return(commitment_id, date)

    if not pref_return_completed:
        print('Preferred Return not completed')
        return False, 0, 0, 0

    date = np.datetime64(date)

    total_catch_up = carried_interest_rate * np.abs(pref_lp_allocation) / (catch_up_rate - carried_interest_rate)

    effective_catch_up = np.min([total_catch_up, pref_return_capital_remaining])

    return (total_catch_up <= pref_return_capital_remaining), 0, effective_catch_up, pref_return_capital_remaining - effective_catch_up

catch_up_completed, catch_up_lp_allocation, catch_up_gp_allocation, next_tier_capital = catch_up(1, '2024-01-02')
print('Catch Up Completed:', catch_up_completed)
print('LP Allocation:', np.abs(catch_up_lp_allocation))
print('GP Allocation:', catch_up_gp_allocation)
print('Next Tier Capital:', next_tier_capital)

Catch Up Completed: True
LP Allocation: 0
GP Allocation: 2401.6533468145717
Next Tier Capital: 137991.73326592715


In [ ]:
def final_split(commitment_id, date):

    roc_completed, roc_lp_allocation, roc_gp_allocation, roc_capital_remaining = return_of_capital(commitment_id, date)
    pref_return_completed, pref_lp_allocation, pref_gp_allocation, pref_return_capital_remaining = preferred_return(commitment_id, date)
    catch_up_completed, catch_up_lp_allocation, catch_up_gp_allocation, catch_up_capital_remaining = catch_up(commitment_id, date)
    
    if not catch_up_completed:
        print('Catch Up not completed')
        return False, 0, 0, 0

    date = np.datetime64(date)

    lp_allocation = catch_up_capital_remaining * (1-carried_interest_rate)
    gp_allocation = catch_up_capital_remaining * (carried_interest_rate)

    return True, lp_allocation, gp_allocation, 0

final_split_completed, final_lp_allocation, final_gp_allocation, next_tier_capital = final_split(1, '2024-01-02')
print('Final Split Completed:', final_split_completed)
print('LP Allocation:', np.abs(final_lp_allocation))
print('GP Allocation:', final_gp_allocation)
print('Next Tier Capital:', next_tier_capital)

Final Split Completed: True
LP Allocation: 110393.38661274173
GP Allocation: 27598.346653185432
Next Tier Capital: 0


In [26]:
# check 
total_lp_distributions = np.abs(roc_lp_allocation) + np.abs(pref_lp_allocation) + np.abs(catch_up_lp_allocation) + np.abs(final_lp_allocation)
total_gp_distributions = np.abs(roc_gp_allocation) + np.abs(pref_gp_allocation) + np.abs(catch_up_gp_allocation) + np.abs(final_gp_allocation)

initial_commitment = df[
    (df['commitment_id'] == 1) &
    (df['transaction_date'] < '2024-01-02') &
    (df['contribution_or_distribution'] == 'contribution')
]['transaction_amount'].sum()

print('Total LP Distributions: ', total_lp_distributions)
print('Total GP Distributions: ', total_gp_distributions)

print('Total Distributions: ', total_lp_distributions + total_gp_distributions, '\n')
print('Net Split: ', (total_lp_distributions + initial_commitment) / (total_lp_distributions + initial_commitment + total_gp_distributions))

Total LP Distributions:  170000.0
Total GP Distributions:  30000.000000000004
Total Distributions:  200000.0 

Net Split:  0.8
